<a href="https://colab.research.google.com/github/devreborges/projetos_analise_dados/blob/main/dashboard_python_preco_combustiveis_rj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dash

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px


In [ ]:
# URL do arquivo no Google Drive
url = 'https://drive.google.com/uc?id=1I8FZ3lexsl_6sirAzyf6pbW0h339kyyE'

df = pd.read_csv(url)

In [ ]:
df_gasolina = df.query('`Produto` == "GASOLINA"')
df_etanol = df.query('`Produto` == "ETANOL"')
df_adtivada = df.query('`Produto` == "GASOLINA ADITIVADA"')

In [ ]:
df_rj = df_gasolina[df_gasolina['Estado - Sigla'] == 'RJ']
df_rj_etanol = df_etanol[df_etanol['Estado - Sigla'] == 'RJ']

In [ ]:
# Crie um aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div(children=[
    html.H1(children='Preço dos Combustíveis'),

    html.Div(id='preco-info'),

    html.Div(children=[
        html.Label('Selecione o Município:'),
        dcc.Dropdown(
            id='municipio-dropdown',
            options=[{'label': municipio, 'value': municipio} for municipio in df_rj['Municipio'].unique()],
            value=df_rj['Municipio'].unique()[0]  # Valor padrão
        ),
    ]),

    html.Div(children=[
        html.Label('Selecione o Bairro:'),
        dcc.Dropdown(
            id='bairro-dropdown',
            options=[],  # As opções serão atualizadas dinamicamente com base no Município selecionado
            value=''  # Valor padrão
        ),
    ]),

    dcc.Graph(id='preco-combustiveis-graph'),

    html.Div(id='preco-bairros-grafico')
])

# Callback para atualizar as opções do seletor de Bairro com base no Município selecionado
@app.callback(
    Output('bairro-dropdown', 'options'),
    [Input('municipio-dropdown', 'value')]
)
def update_bairros_options(selected_municipio):
    if selected_municipio:
        bairros_options = [{'label': bairro, 'value': bairro} for bairro in df_rj[df_rj['Municipio'] == selected_municipio]['Bairro'].unique()]
    else:
        bairros_options = []
    return bairros_options

# Callback para atualizar o texto informativo
@app.callback(
    Output('preco-info', 'children'),
    [Input('municipio-dropdown', 'value'),
     Input('bairro-dropdown', 'value')]
)
def update_preco_info(selected_municipio, selected_bairro):
    if selected_municipio:
        filtered_df = df_rj[df_rj['Municipio'] == selected_municipio]
        menor_preco = filtered_df['Valor de Venda'].min()
        maior_preco = filtered_df['Valor de Venda'].max()
        texto_menor = f'O menor valor da gasolina no município "{selected_municipio}" é {menor_preco:.2f}'
        texto_maior = f'O maior valor da gasolina no município "{selected_municipio}" é {maior_preco:.2f}'
    else:
        texto_menor = ''
        texto_maior = ''
    return [html.P(texto_menor), html.P(texto_maior)]


# Callback para atualizar o gráfico com base nos seletores de Município e Bairro selecionados
@app.callback(
    Output('preco-combustiveis-graph', 'figure'),
    [Input('municipio-dropdown', 'value'),
     Input('bairro-dropdown', 'value')]
)
def update_graph(selected_municipio, selected_bairro):
    filtered_df = df_rj
    if selected_municipio:
        filtered_df = filtered_df[filtered_df['Municipio'] == selected_municipio]
    if selected_bairro:
        filtered_df = filtered_df[filtered_df['Bairro'] == selected_bairro]

    menor_preco = filtered_df['Valor de Venda'].min()
    maior_preco = filtered_df['Valor de Venda'].max()
    preco_medio = filtered_df['Valor de Venda'].mean()

    fig = px.bar(
        x=['Menor Preço', 'Maior Preço', 'Preço Médio'],
        y=[menor_preco, preco_medio, maior_preco],
        labels={'x': 'Tipo de Preço', 'y': 'Valor de Venda'},
        title='Preços dos Combustíveis'
    )

    # Definir cores das barras
    fig.update_traces(marker_color=['yellow', 'orange', 'red'], text=[f'{menor_preco:.2f}', f'{preco_medio:.2f}', f'{maior_preco:.2f}'], textposition='outside')

    # Ajustar o tamanho do gráfico
    fig.update_layout(width=400, height=600)

    return fig

# Callback para atualizar o gráfico de barras com os 10 preços mais baixos de cada município selecionado
@app.callback(
    Output('preco-bairros-grafico', 'children'),
    [Input('municipio-dropdown', 'value')]
)
def update_bairros_graph(selected_municipio):
    if selected_municipio:
        # Filtrar o DataFrame para o município selecionado
        filtered_df = df_rj[df_rj['Municipio'] == selected_municipio]
        # Ordenar o DataFrame pelos preços de venda
        sorted_df = filtered_df.sort_values(by='Valor de Venda')
        # Selecionar os 10 preços mais baixos
        top_10_prices = sorted_df.head(10)

        # Criar o gráfico de barras
        fig = px.bar(
            top_10_prices,
            y='Bairro',
            x='Valor de Venda',
            orientation='h',
            labels={'x': 'Valor de Venda', 'y': 'Bairro'},
            title=f'10 Menores Preços de Combustíveis em {selected_municipio}',
            color='Valor de Venda',
            color_continuous_scale='viridis',  # Esquema de cores (de amarelo para vermelho)
            text='Valor de Venda',
            width=600,
            height=800
        )

        return dcc.Graph(figure=fig)

    else:
        return html.Div()



# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>